In [10]:
from datasets import load_dataset
from llama_index.core import Document, VectorStoreIndex
from llama_index.core.ingestion import IngestionPipeline
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.evaluation import RelevancyEvaluator
from pinecone import Pinecone, ServerlessSpec
import re, os, nest_asyncio
from time import sleep

In [2]:
from dotenv import load_dotenv
load_dotenv()

pinecone_api_key = os.environ.get('PINECONE_API_KEY')
openai_api_key = os.environ.get('OPENAI_API_KEY')

In [3]:
# 실습 데이터 로드 및 판다스 데이터프레임화
dataset = load_dataset("lcw99/wikipedia-korean-20221001", split='train[:100]')
data = dataset.to_pandas()[['id', 'text', 'title']].drop_duplicates(subset='text', keep='first')

In [4]:
# 스트링 전처리
def clean_up_text(content: str) -> str:
    content = re.sub(r'(\w+)-\n(\w+)', r'\1\2', content)
    content = re.sub(r'\\n|  —|——————————|—————————|—————|\\u[\dA-Fa-f]{4}|\uf075|\uf0b7', "", content)
    content = re.sub(r'(\w)\s*-\s*(\w)', r'\1-\2', content)
    content = re.sub(r'\s+', ' ', content)

    return content

# 텍스트 전처리와 곁들여 document화 진행
docs = [Document(
    text=clean_up_text(row['text']),
    doc_id=row['id'],
    extra_info={'title': row['title']}
) for _, row in data.iterrows()]

In [7]:
# docs[0]

In [8]:
# Pinecone setup

pc = Pinecone(api_key=pinecone_api_key)

In [11]:
# 파인콘 인덱스 생성
from time import sleep

index_name="quickstart"

if index_name in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.delete_index(index_name)

pc.create_index(
   name=index_name,
   dimension=1536,
   metric="dotproduct",
   spec=ServerlessSpec(
       cloud='aws',
       region='us-east-1'
   )
)

while not pc.describe_index(index_name).status['ready']:
    sleep(1)
index = pc.Index(index_name)
sleep(1)
index_stats = index.describe_index_stats()
print(index_stats)

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


In [ ]:
# Ingestion Pipeline 정의
# chunking, embedding 한번에 수행
embed_model = OpenAIEmbedding(api_key=openai_api_key, model='text-embedding-ada-002')
vector_store = PineconeVectorStore(pinecone_index=index)
pipeline = IngestionPipeline(transformations=[SimpleNodeParser(), embed_model], vector_store=vector_store)

# Run pipeline
pipeline.run(documents=docs)

In [13]:
# querying stage 구성
vector_index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
retriever = VectorIndexRetriever(index=vector_index, similarty_top_k=5)

In [ ]:
# 단순 retreive 테스트
answer = retriever.retrieve('구글의 광고수익에 대해 알려줘')
print([i.get_content() for i in answer])

In [19]:
from llama_index.core import ServiceContext

# RAG 구성
query_engine = RetrieverQueryEngine(retriever=retriever).from_args(retriever=retriever)
nest_asyncio.apply() # 다중 query 수행시 필요
 
query = '구글의 광고수익에 대해 알려줘'
llm_response = query_engine.query(query)
llm_response.response

'구글의 광고 수익은 2008년에는 42억 달러로 상승하여 전체 수입의 97%를 占하였고, 2011년에는 미국 내 웹 광고 시장의 전체 규모가 600억 달러에 이를 것으로 전망되었다.'

In [20]:
# 구체적인 수치에 대한 질문
query = '구글의 2008년 광고 수익이 얼마였지?'
llm_response = query_engine.query(query)
llm_response.response

'구글의 2008년 광고 수익은 42억 달러였습니다.'

In [21]:
# 명시되지 않았지만, 1차적인 reasoning이 있으면 답변 가능한 질문
query = '2008년 CBS, NBC, ABC, FOX, CW의 합산 광고 수익이 달러로 얼마야?'
llm_response = query_engine.query(query)
llm_response.response

'The combined advertising revenue of CBS, NBC, ABC, FOX, and CW in 2008 was equivalent to the advertising revenue of Google in that year, which was over 42 billion dollars.'

In [22]:
# 답할수 없는 질문일땐?
query = '네이버의 광고수익에 대해 알려줘'
llm_response = query_engine.query(query)
llm_response.response

'네이버의 광고수익은 구글과는 다르게 명시적으로 언급되지 않았습니다. 따라서 네이버의 광고수익에 대한 구체적인 정보는 이 문서에서는 제공되지 않습니다.'

In [ ]:
# 답변 생성시 사용된 source node 확인
llm_response_source_nodes = [i.get_content() for i in llm_response.source_nodes]
llm_response_source_nodes

In [23]:
# 할루시네이션 답변을 평가하고 사용자에게 노출 전 평가
query = '구글의 2008년 광고 수익이 얼마였지?'
evaluator = RelevancyEvaluator()
eval_result = evaluator.evaluate_response(query=query, response=llm_response)

llm_response_source_nodes = [i.get_content() for i in llm_response.source_nodes]
print(f'\nGiven the {len(llm_response_source_nodes)} chunks of content (below), is your LLM\'s response relevant? {eval_result.passing}\n \
        \n ----Contexts----- \n \
        \n{llm_response_source_nodes}')


Given the 2 chunks of content (below), is your LLM's response relevant? True
         
 ----Contexts----- 
         
['그리고 그 가운데 97%가 광고 수입이었다. 2008년, 구글의 광고 수입은 5개 방송사(CBS, NBC, ABC, FOX, CW)의 광고 수입을 합한 것에 맞먹었다. 2011년에 이르면 미국 내 웹 광고는 600억 달러(전체 13%)에 달할 것으로 전망된다. 게다가 구글은 TV, 라디오, 신문에 광고를 판매함으로써 시장점유율을 가일층 확대할 사업구상을 이미 개시했다. 사용자가 텍스트 광고를 클릭할 때만 광고료를 부과해서 광고주들 중에서 우군을 확보했고, 무료이자 2009년 초반까지 광고가 붙지 않았던 구글 뉴스로 뉴스독자들 중에서 우군을 확보했으며, 광고 수익과 신규 고객을 발생시켜 줌으로써 웹사이트와 소규모 사업자들 중에서 우군을 확보했다. 구글은 두 번째 경매 프로그램 애드센스 때부터 수입의 20%만 자기 주머니에 넣고 나머지는 웹사이트들에게 돌려 주었다. 2008년에 구글은 총 50억 달러가 넘는 돈을 수십만에 달하는 \'파트너들\'에게 제공했다. 제품 Gmail, 구글 뉴스, 구글 어스, 구글 맵스, 구글 비디오, 구글 번역, 피카사(Picasa-디지털 사진 공유), 구글 클래스룸, 구글 북스(발행된 모든 책 검색), 구글 트렌드 (검색량 통계 제공), 오컷(Orkut-인맥, 친목 사이트), 여기에 데스크톱(Desktop)이나 문서도구(Docs), 구글 플레이같은 \'클라우드 컴퓨팅(cloud computing)\' 응용 프로그램까지 제공한다. 구글에서 사용하는 컴퓨터는 보통 PC들로 구성된 컴퓨터 클러스터들인데, 이 클러스터들은 일을 병렬적으로 처리하여 방대한 양의 데이터베이스를 처리한다. 특히 여러 대의 PC를 운영하면서 계속적인 데이터베이스를 처리하기 위해 한 컴퓨터에 오류가 났을 경우 그 컴퓨터는 꺼지고, 다른 컴퓨터가 일을 계속 처